In [12]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

In [13]:
query_simtest = """
SELECT
  trim('"' FROM block::text )::int as block,
  trim('"' FROM min::text )::int as min,
  s.holding,
  s.acks,
  s.msgqueue,
  s.inmsgqueue,
  s.apiqueue,
  s.ackqueue,
  s.timermsg,
  ts::text::time
FROM (SELECT
  --mt     | "LIST_SIZES Holding: %v, Acks: %v, MsgQueue: %v, InMsgQueue: %v, APIQueue: %v, AckQueue: %v, TimerMsg: %v "
  run,
  e->'log'->'height' as block,
  e->'log'->'min' as min,
  e->'log'->'event'->0 as holding,
  e->'log'->'event'->1 as acks,
  e->'log'->'event'->2 as msgqueue,
  e->'log'->'event'->3 as inmsgqueue,
  e->'log'->'event'->4 as apiqueue,
  e->'log'->'event'->5 as ackqueue,
  e->'log'->'event'->6 as timermsg,
  e->'log'->'event'->7 as fmt,
  e->'ts' as ts
  --e as l
FROM
  logs
) as s

WHERE
  s.fmt::varchar like '%LIST_SIZES%'
ORDER BY ts;

"""

connection = pg.connect("host='localdb' dbname=load user=load password='load'")


In [14]:
df = pd.read_sql_query(query_simtest,con=connection)
print(df)

      block  min  holding  acks  msgqueue  inmsgqueue  apiqueue  ackqueue  \
0         5    1      185     0        14           0        32         0   
1         5    1        0     0         0           0         0         0   
2         5    1        0     0         0           0         0         0   
3         5    1      377     0         0          21         0         0   
4         5    1       28     0         0           0         0         0   
5         5    1       71     0         0          19         0         4   
6         5    1      327     0         0          51         0         0   
7         5    1       50     0         0          17         0         0   
8         5    1       50     0         0           0         0         0   
9         5    2      246     0         9           0         0         1   
10        5    2      425     0         0           0         0         0   
11        5    2      214     1         0           0         0         0   

In [16]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd
from datetime import datetime

data = [
    go.Scatter(x=df['ts'], y=df['holding'], name='holding'),
    go.Scatter(x=df['ts'], y=df['acks'], name='acks'),
    go.Scatter(x=df['ts'], y=df['msgqueue'], name='msgqueue'),
    go.Scatter(x=df['ts'], y=df['inmsgqueue'], name='inmsgqueue'),
]

py.iplot(data, filename = 'holding_msg_queues_comp_with_delay')